# Natural Language Processing and Information Extraction
## Deep learning - Practical Session

__Nov 7, 2025__

__Varvara Arzt__

Material of this lecture was created by Ádám Kovács and extended by Varvara Arzt

<a href="https://colab.research.google.com/github/tuw-nlp-ie/tuw-nlp-ie-2025WS/blob/main/lectures/05_Deep_learning_practical_lesson/deep_learning_practical_lesson_without_outputs.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>



## Dataset Overview:
In this lecture, we are going to work with a classification dataset from [SemEval 2019 - Task 6](https://github.com/ZeyadZanaty/offenseval), which focuses on identifying and categorising Offensive Language in Social Media.

## About SemEval:
SemEval (Semantic Evaluation) is a series of international NLP research workshops where researchers compete to solve real-world language problems. Each year, organisers release datasets and challenges (called "shared tasks"), and teams from around the world build models to tackle these problems and compare their results. [Here](https://github.com/SemEval/SemEval2025/blob/main/tasks.md) you can find a list of shared tasks for SemEval-2025.

## Tools and Setup

### PyTorch

We'll be using [PyTorch](https://pytorch.org/docs/stable/index.html), an open-source library for building deep learning models that can run efficiently on GPUs. It's one of the most widely used frameworks for neural networks in both research and industry.

For your projects, you may want to explore **[HuggingFace](https://huggingface.co)** as well, a library that provides access to state-of-the-art pre-trained models and makes it much easier to work with modern transformer architectures.

### Prerequisites:

**Dataset:**
- SemEval 2019 - Task 6 dataset (code for downloading will be provided)

**Required Libraries:**

- PyTorch, pandas, scikit-learn, spacy
- transformers, accelerate, safetensors (for HuggingFace models)
- wandb (for experiment tracking)

### Installation

Run the following commands to install all required packages:
```python
# Install required packages
!pip install -q torch pandas scikit-learn spacy transformers accelerate safetensors wandb emoji==0.6.0

# Download spacy language model
!python -m spacy download en_core_web_sm
```

For OS-specific or CUDA-enabled PyTorch installation, consult the [PyTorch installation guide](https://pytorch.org/get-started/locally/).

> **Note:** We strongly recommend using [Google Colab](https://colab.research.google.com/) for this lab. It provides free GPU access and ensures a consistent environment for everyone, making it easier to reproduce results.

## What We'll Cover

In this lecture, we'll explore different approaches for text classification, progressing from simple to more advanced models:

1. **Feed Forward Neural Network (FNN)** with Bag-of-Words representation
2. **Multi-layer FNNs** for deeper feature learning
3. **Neural Networks with embedding layers** (+ max-pooling)
4. **Recurrent Neural Networks (RNNs)** and **LSTM networks** for sequential processing
5. **Transformer architecture (BERT)** for state-of-the-art performance

## 1. Setup and Installation

In [ ]:
# Install required packages
!pip install -q torch pandas scikit-learn spacy transformers accelerate safetensors wandb
!pip3 install emoji==0.6.0
!python -m spacy download en_core_web_sm

In [ ]:
import os
import time
import warnings
import urllib.request
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import spacy

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

from transformers import (
    BertTokenizer,
    AutoTokenizer, 
    BertForSequenceClassification,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
import wandb

# Configuration
warnings.filterwarnings('ignore')
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
np.random.seed(SEED)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## 2. Data Loading and Preprocessing

In [ ]:
# Download dataset
os.makedirs("./data", exist_ok=True)

dataset_url = "https://raw.githubusercontent.com/ZeyadZanaty/offenseval/master/datasets/training-v1/offenseval-training-v1.tsv"
dataset_path = "data/offenseval.tsv"

if not os.path.exists(dataset_path):
    urllib.request.urlretrieve(dataset_url, dataset_path)
    print("Dataset downloaded successfully")
else:
    print("Dataset already exists")

### Read in the dataset into a Pandas DataFrame
Load the dataset into a Pandas DataFrame using `pd.read_csv()`. The dataset is tab-separated and contains 5 columns:
`id`, `tweet`, `subtask_a`, `subtask_b`, `subtask_c`.

In [ ]:
def read_dataset():
    train_data = pd.read_csv("./data/offenseval.tsv", sep="\t")
    return train_data

In [ ]:
train_data_unprocessed = read_dataset()
train_data_unprocessed

### Convert `subtask_a` Labels to Binary Format
The task is to classify tweets as either offensive (OFF) or not offensive (NOT). For machine learning algorithms, you will need integer labels instead of strings. Create a new column called `label` where:
- `NOT` → 0 (not offensive)
- `OFF` → 1 (offensive)

In [ ]:
def transform(train_data):
    labels = {"NOT": 0, "OFF": 1}

    train_data["label"] = [labels[item] for item in train_data.subtask_a]
    train_data["tweet"] = train_data["tweet"].str.replace("@USER", "")

    return train_data

In [ ]:
train_data = transform(train_data_unprocessed)
train_data

In [ ]:
train_data.groupby("subtask_a").size()

### Split the dataset into a train and a validation dataset
Use the random seed for splitting. You should split the dataset into 70% training data and 30% validation data

In [ ]:
def split_data(train_data, random_seed):
    tr_data, val_data = train_test_split(train_data, test_size=0.3, random_state=SEED)
    return tr_data, val_data

In [ ]:
tr_data, val_data = split_data(train_data, SEED)

## 3. Text Preprocessing with CountVectorizer

We'll use [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) from scikit-learn to convert text into numerical features. This tool will:

- Tokenize, lowercase the text
- Filter out stopwords
- Convert the text into one-hot encoded vectors
- Select the _n_-best features

We fit CountVectorizer using _3000_ features (vocabulary size)

We will also _lemmatize_ texts using the _spaCy_ package and its lemmatizer. Check the [docs](https://spacy.io/models/en) for more.

CountVectorizer transforms text into a Bag-of-Words representation by counting how many times each word appears in a document. This creates a document-term matrix where each row is a document and each column is a word from the vocabulary.

![Bag-of-Words visualization](https://miro.medium.com/v2/resize:fit:4800/format:webp/1*3IACMnNpwVlCl8kSTJocPA.png)

In [ ]:
# Load spaCy model (disable unused components for speed)
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

class SpacyLemmaTokenizer:
    """Custom tokenizer that lemmatizes words using spaCy."""
    def __init__(self):
        self.nlp = nlp
    
    def __call__(self, doc):
        return [token.lemma_ for token in self.nlp(doc)]


def prepare_vectorizer(tr_data):
    """Initialize and fit CountVectorizer on training data."""
    vectorizer = CountVectorizer(
        max_features=3000,
        tokenizer=SpacyLemmaTokenizer(),
        stop_words="english"
    )

    word_to_ix = vectorizer.fit(tr_data.tweet)

    return word_to_ix

In [ ]:
# Fit vectorizer on training data
word_to_ix = prepare_vectorizer(tr_data)
VOCAB_SIZE = len(word_to_ix.vocabulary_)
assert VOCAB_SIZE == 3000

**Example: Lemmatization with spaCy**

In [ ]:
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
sentence = "the kids are running happily towards the cars"
doc = nlp(sentence)

print("Original:", [token.text for token in doc])
print("Lemmatized:", [token.lemma_ for token in doc])

**View top 10 most frequent words in our dataset after preprocessing**

In [ ]:
# Transform training data to Bag-of-Words representation
X = word_to_ix.transform(tr_data.tweet)

# Top 10 words
top_10 = sorted([(word, X.sum(axis=0).A1[idx]) for word, idx in word_to_ix.vocabulary_.items()], 
                key=lambda x: x[1], reverse=True)[:10]

print("Top 10 words:", top_10)

**Example: Transform a new sentence to a Bag-of-Words representation**

In [ ]:
example = word_to_ix.transform(["he loves science"]).toarray()
print(f"Bag-of-words vector: '{example}':")
print(f"Vector shape: {example.shape}")
print(f"Non-zero positions: {example.nonzero()}")

## 4. Prepare the DataLoader for Batch Processing

The `prepare_dataloader()` function converts our text data into PyTorch tensors:

1. **Transform tweets** into one-hot encoded vectors using CountVectorizer
2. **Create tensors** from the vectors and labels (FloatTensor for features, LongTensor for labels)
3. **Move tensors to device** (GPU/CPU) - all tensors must be on the same device for training
4. **Zip together** features and labels as a list of tuples

In [ ]:
# PyTorch operates on Tensors - optimized data structures for training neural networks
# Learn more: https://pytorch.org/docs/stable/tensors.html
def prepare_dataloader(tr_data, val_data, word_to_ix):
    """
    Convert text data to PyTorch tensors.
    
    Args:
        tr_data: Training dataset
        val_data: Validation dataset
        word_to_ix: Fitted CountVectorizer
    
    Returns:
        tr_data_loader: List of (features, label) tuples for training
        val_data_loader: List of (features, label) tuples for validation
    """
    # Transform text to one-hot encoded vectors and convert to tensors
    tr_data_vecs = torch.FloatTensor(word_to_ix.transform(tr_data.tweet).toarray()).to(
        device
    )
    tr_labels = torch.LongTensor(tr_data.label.tolist()).to(device)

    val_data_vecs = torch.FloatTensor(
        word_to_ix.transform(val_data.tweet).toarray()
    ).to(device)
    val_labels = torch.LongTensor(val_data.label.tolist()).to(device)

    # Create list of (sample, label) tuples
    tr_data_loader = [(sample, label) for sample, label in zip(tr_data_vecs, tr_labels)]
    val_data_loader = [
        (sample, label) for sample, label in zip(val_data_vecs, val_labels)
    ]

    return tr_data_loader, val_data_loader

In [ ]:
tr_data_loader, val_data_loader = prepare_dataloader(tr_data, val_data, word_to_ix)

> **Note:** Our current approach loads and transforms the entire dataset into memory at once. This works perfectly for smaller datasets like OffensEval (~13,000 tweets). For larger datasets that don't fit in memory (e.g., millions of images or documents), you would use PyTorch's custom `torch.utils.data.Dataset` class, which loads and processes data on-the-fly as each batch is needed during training. See the [documentation](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html) for examples.

### Create DataLoader Objects for Batch Processing

Now we'll wrap our data in PyTorch's `DataLoader` class, which handles batching and shuffling automatically.

**Why batching?** We don't feed the entire dataset to the model at once. Instead, we process small batches (e.g., 64 samples) at a time, which:
- Reduces memory usage
- Provides more frequent weight updates
- Often leads to better model generalization

In [ ]:
# Batch size: number of samples processed before updating model weights
# Try experimenting with different values (8, 16, 32, 64, 128) to see the impact on performance
BATCH_SIZE = 64

In [ ]:
def create_dataloader_iterators(tr_data_loader, val_data_loader, BATCH_SIZE):
    """
    Create DataLoader objects for batched training.
    
    Args:
        tr_data_loader: List of training (features, label) tuples
        val_data_loader: List of validation (features, label) tuples
        batch_size: Number of samples per batch
    
    Returns:
        train_iterator: DataLoader for training (shuffled)
        valid_iterator: DataLoader for validation (not shuffled)
    """
    # Shuffle training data each epoch to prevent the model from learning order patterns
    # Don't shuffle validation data - we want consistent evaluation
    
    train_iterator = DataLoader(
        tr_data_loader,
        batch_size=BATCH_SIZE,
        shuffle=True,
    )

    valid_iterator = DataLoader(
        val_data_loader,
        batch_size=BATCH_SIZE,
        shuffle=False,
    )

    return train_iterator, valid_iterator

In [ ]:
train_iterator, valid_iterator = create_dataloader_iterators(
    tr_data_loader, val_data_loader, BATCH_SIZE
)
assert type(train_iterator) == torch.utils.data.dataloader.DataLoader

In [ ]:
print(f"Number of training batches: {len(train_iterator)}")
print(f"Number of validation batches: {len(valid_iterator)}")

## 5. Building the Model

### Simple Bag-of-Words Classifier

Our first model is a simple single-layer neural network:
- **Input:** Bag-of-Words vectors (size = vocabulary size)
- **Output:** Class probabilities (2 classes: offensive/not offensive)
- **Architecture:** One linear transformation followed by softmax activation

In [ ]:
class BoWClassifier(nn.Module):
    """
    Simple Bag-of-Words classifier with one linear layer.
    
    Note: All PyTorch models must inherit from nn.Module.
    This is a 'shallow' network with just one layer - we'll build deeper ones later!
    """
    def __init__(self, vocab_size, num_labels):
        # Always call the parent class constructor in PyTorch models
        # This initializes important internal mechanisms
        super(BoWClassifier, self).__init__()

        # Single linear layer: applies transformation y = xW^T + b
        # To make this "deep", add more nn.Linear layers with activation functions
        # (e.g., ReLU) between them. For now, this shallow architecture serves as our baseline.
        self.linear = nn.Linear(vocab_size, num_labels)

    def forward(self, bow_vec, sequence_lens=None):
        """
        Forward pass through the network.
        
        Args:
            bow_vec: Bag-of-Words input vectors [batch_size, vocab_size]
            sequence_lens: Not used in this model, but included for compatibility 
                          with later models (RNN/LSTM) that need sequence lengths
        
        Returns:
            Log probabilities for each class [batch_size, num_labels]
        """
        # Pass input through the linear layer
        output = self.linear(bow_vec)
        # Apply log_softmax to convert raw scores into log probabilities
        # Softmax creates a probability distribution (values sum to 1)
        # log_softmax is more stable than softmax and works with NLLLoss
        # dim=1 means we apply softmax across the class dimension, not the batch dim
        return nn.functional.log_softmax(output, dim=1)

In [ ]:
INPUT_DIM = VOCAB_SIZE # Size of our vocabulary
OUTPUT_DIM = 2 # Binary classification: offensive vs not offensive

In [ ]:
# Initialize model
model = BoWClassifier(INPUT_DIM, OUTPUT_DIM)
print(model)
print(f"Model parameters: {sum(p.numel() for p in model.parameters())} weights")

**Explore the model weights:**

In [ ]:
# The model starts with random weights
print(f"Weight matrix shape: {model.linear.weight.shape}")
print(f"First few weights:\n{model.linear.weight[:2, :5]}")

### Set Up Optimizer and Loss Function

In [ ]:
# Optimizer: updates model weights based on gradients
# Adam is a popular choice that adapts the learning rate automatically
optimizer = optim.Adam(model.parameters(), lr=1e-3)
# Loss function: Negative Log Likelihood Loss (works with log_softmax output)
criterion = nn.NLLLoss()

In [ ]:
# Move model and loss function to the correct device (GPU/CPU)
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
assert model.linear.out_features == 2

## Training and Evaluation Functions

### Performance Calculation

### Training and evaluating PyTorch models
- __calculate_performance__: This should calculate the batch-wise precision, recall, and fscore of your model!
- __train__ - Train your model on the training data! This function should set the model to training mode, then use the given iterator to iterate through the training samples and make predictions using the provided model. You should then propagate back the error with the loss function and the optimizer. Finally return the average epoch loss and performance!
- __evaluate__ - Evaluate your model on the validation dataset. This function is essentially the same as the trainnig function, but you should set your model to eval mode and don't propagate back the errors to your weights!

In [ ]:
def calculate_performance(preds, y):
    """
    Calculate precision, recall, and F-score for predictions.
    
    Args:
        preds: Model predictions (log probabilities)
        y: True labels
    
    Returns:
        precision, recall, fscore for the positive class (offensive)
    """
    # Get predicted class (highest probability)
    rounded_preds = preds.argmax(1)

    # Calculate the correct predictions batch-wise and calculate precision, recall, and fscore
    # WARNING: Tensors here could be on the GPU, so make sure to copy everything to CPU
    precision, recall, fscore, support = precision_recall_fscore_support(
        rounded_preds.cpu(), y.cpu()
    )

    return precision[1], recall[1], fscore[1]

### Training Function

In [ ]:
def train(model, iterator, optimizer, criterion):
    """
    Train the model for one epoch.
    
    Args:
        model: Neural network model
        iterator: DataLoader for training data
        optimizer: Optimizer for updating weights
        criterion: Loss function
    
    Returns:
        Average loss, precision, recall, and F-score for the epoch
    """
    epoch_loss = 0
    epoch_prec = 0
    epoch_recall = 0
    epoch_fscore = 0

    # Set model to training mode (enables gradient computation)
    model.train()

    # We calculate the error on batches so the iterator will return matrices with shape [BATCH_SIZE, VOCAB_SIZE]
    for batch in iterator:
        text_vecs = batch[0] # Input features
        labels = batch[1]   # True labels
        sen_lens = []
        texts = []

        # Handle additional batch elements for later models
        if len(batch) > 2:
            sen_lens = batch[2]
            texts = batch[3]

        # Reset gradients from previous step
        optimizer.zero_grad()

        # Forward pass: compute predictions
        predictions = model(text_vecs, sen_lens)

        # Calculate loss on  predictions (predictions are log probabilities)
        loss = criterion(predictions, labels)

        # Calculate performance metrics
        prec, recall, fscore = calculate_performance(predictions, labels)

        # Backward pass: compute gradients
        loss.backward()
        # Update model weights
        optimizer.step()

        # Add batch-wise loss to the epoch-wise loss
        epoch_loss += loss.item()
        # Do the same with the performance metrics
        epoch_prec += prec.item()
        epoch_recall += recall.item()
        epoch_fscore += fscore.item()
    # Return average metrics over all batches
    return (
        epoch_loss / len(iterator),
        epoch_prec / len(iterator),
        epoch_recall / len(iterator),
        epoch_fscore / len(iterator),
    )

### Evaluation Function

In [ ]:
# The evaluation is done on the validation dataset
def evaluate(model, iterator, criterion):
    """
    Evaluate the model on validation data.
    
    Args:
        model: Neural network model
        iterator: DataLoader for validation data
        criterion: Loss function
    
    Returns:
        Average loss, precision, recall, and F-score for the validation set
    """
    epoch_loss = 0
    epoch_prec = 0
    epoch_recall = 0
    epoch_fscore = 0
    # Set model to evaluation mode (disables dropout, batchnorm updates, etc.)
    model.eval()

    # Disable gradient computation (saves memory and speeds up evaluation)
    with torch.no_grad():
        # The remaining part is the same with the difference of not using the optimizer to backpropagation
        for batch in iterator:
            text_vecs = batch[0]
            labels = batch[1]
            sen_lens = []
            texts = []

            if len(batch) > 2:
                sen_lens = batch[2]
                texts = batch[3]

            # Forward pass only (no backward pass)
            predictions = model(text_vecs, sen_lens)
            loss = criterion(predictions, labels)

            prec, recall, fscore = calculate_performance(predictions, labels)

            epoch_loss += loss.item()
            epoch_prec += prec.item()
            epoch_recall += recall.item()
            epoch_fscore += fscore.item()

    # Return average metrics on the whole epoch
    return (
        epoch_loss / len(iterator),
        epoch_prec / len(iterator),
        epoch_recall / len(iterator),
        epoch_fscore / len(iterator),
    )

### Helper Functions

In [ ]:
def epoch_time(start_time, end_time):
    """Calculate and format training time."""
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
def plot_losses(train_losses, valid_losses, n_epochs):
    """Plot training and validation losses over epochs."""
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, n_epochs + 1), train_losses, label="Training Loss")
    plt.plot(range(1, n_epochs + 1), valid_losses, label="Validation Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Training and Validation Loss Over Epochs")
    plt.legend()
    plt.grid()
    plt.show()

## Training Loop

Now let's train the model! The training loop will:
1. Train on the training set for each epoch
2. Evaluate on the validation set
3. Save the best model (lowest validation loss)
4. Plot the learning curves

In [ ]:
def training_loop(epoch_number=15):
    """
    Main training loop.
    
    Args:
        epoch_number: Number of epochs to train
    """
    N_EPOCHS = epoch_number

    best_valid_loss = float("inf")

    # Lists to store loss values for plotting
    train_losses = []
    valid_losses = []

    # We loop forward on the epoch number
    for epoch in range(N_EPOCHS):

        start_time = time.time()

        # Train the model on the training set using the dataloader for one epoch
        train_loss, train_prec, train_rec, train_fscore = train(
            model, train_iterator, optimizer, criterion
        )
        # Evaluate on validation set
        valid_loss, valid_prec, valid_rec, valid_fscore = evaluate(
            model, valid_iterator, criterion
        )

        end_time = time.time()

        epoch_mins, epoch_secs = epoch_time(start_time, end_time)

        # Store losses
        train_losses.append(train_loss)
        valid_losses.append(valid_loss)

        # Save best model
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), "tut1-model.pt")
            
        # Print progress
        print(f"Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s")
        print(
            f"\tTrain Loss: {train_loss:.3f} | Train Prec: {train_prec*100:.2f}% | Train Rec: {train_rec*100:.2f}% | Train Fscore: {train_fscore*100:.2f}%"
        )
        print(
            f"\t Val. Loss: {valid_loss:.3f} |  Val Prec: {valid_prec*100:.2f}% | Val Rec: {valid_rec*100:.2f}% | Val Fscore: {valid_fscore*100:.2f}%"
        )
    plot_losses(train_losses, valid_losses, N_EPOCHS)


In [ ]:
# Train the model
# Try different epoch numbers to find the sweet spot between underfitting and overfitting
training_loop(epoch_number=20)

## Important: Model Reinitialization

> **Critical:** If you want to train the model from scratch multiple times, you **must** reinitialize it. Otherwise, training will continue from where it left off rather than starting fresh with random weights.

To reinitialize the model, rerun these lines:
```python
# Reinitialize model with random weights
model = BoWClassifier(INPUT_DIM, OUTPUT_DIM)

# Reinitialize optimizer (needs to track the new model's parameters)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Reinitialize loss function
criterion = nn.NLLLoss()

# Move to device
model = model.to(device)
criterion = criterion.to(device)
```

Or create a helper function:

In [ ]:
def reinitialize(model):
    """Reinitialize model, optimizer, and loss function."""
    # Create new model instance
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.NLLLoss()
    model = model.to(device)
    criterion = criterion.to(device)

In [ ]:
reinitialize(BoWClassifier(INPUT_DIM, OUTPUT_DIM))

## 6. Building a Deeper Network

### Adding Hidden Layers

Our current model has only one layer. Let's make it "deep" by adding:
- **Hidden layer:** An intermediate representation between input and output
- **ReLU activation:** A non-linear function that allows the network to learn complex patterns
- **Multiple transformations:** Input → Hidden → Output

**Why add depth?**
- Single layers can only learn linear relationships
- Hidden layers with activations enable learning non-linear patterns
- This often improves performance on complex tasks



Learn more:
- [ReLU activation](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html)
```

In [ ]:
class BoWDeepClassifier(nn.Module):
    def __init__(self, num_labels, vocab_size, hidden_size):
        """Deep Bag-of-Words classifier with one hidden layer."""

        super(BoWDeepClassifier, self).__init__()
        # First layer: transform from vocabulary size to hidden size
        self.linear1 = nn.Linear(vocab_size, hidden_size)
        #Non-linear activation function
        # ReLU(x) = max(0, x) - introduces non-linearity between layers
        # Without this, multiple linear layers would collapse into a single linear transformation
        self.relu = torch.nn.ReLU()
        # Second layer: transform from hidden size to number of classes
        self.linear2 = nn.Linear(hidden_size, num_labels)

    def forward(self, bow_vec, sequence_lens=None):
        """
        Forward pass through the network.
        
        Args:
            bow_vec: Input Bag-of-Words vectors
            sequence_lens: Not used (kept for compatibility)
        
        Returns:
            Log probabilities for each class
        """
        output = self.linear1(bow_vec)
        output = self.relu(output)
        output = self.linear2(output)

        # Get the log probabilities
        return nn.functional.log_softmax(output, dim=1)

### Configure Hyperparameters


In [ ]:
HIDDEN_SIZE = 200 # Size of the hidden layer - try experimenting with this!
learning_rate = 0.001
BATCH_SIZE = 64
N_EPOCHS = 15

### Initialize and Train

In [ ]:
model = BoWDeepClassifier(OUTPUT_DIM, INPUT_DIM, HIDDEN_SIZE) # initialize the model

# Set up optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.NLLLoss()

# Move to device
model = model.to(device)
criterion = criterion.to(device)

# Display model architecture
print(model)
print(f"Total parameters: {sum(p.numel() for p in model.parameters())}")

> **Experiment:** Try different values for `HIDDEN_SIZE` (e.g., 50, 100, 200, 500) and see how it affects performance. Larger hidden sizes can learn more complex patterns but may overfit on small datasets.Retry

In [ ]:
training_loop()

## 7. Early Stopping to Prevent Overfitting

### What is Early Stopping?

Early stopping is a simple but effective technique to prevent overfitting. The idea:
- Monitor validation loss during training
- If validation loss stops improving for several consecutive epochs (while training loss may still decrease), stop training
- This prevents the model from memorizing the training data

**Why it works:** When validation loss stops improving but training loss keeps decreasing, the model is likely starting to overfit to the training data rather than learning generalizable patterns.

### Implementation

We'll implement a patience-based early stopping:
- **Patience:** Number of epochs to wait for improvement before stopping
- If validation loss doesn't improve for `patience` consecutive epochs, training stops automatically
- We save the best model (lowest validation loss) during training

In [ ]:
# Reinitialize model for fresh training
model = BoWDeepClassifier(OUTPUT_DIM, INPUT_DIM, HIDDEN_SIZE)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.NLLLoss()

model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def training_loop(epoch_number=15, patience=3):
    """
    Training loop with early stopping.
    
    Args:
        epoch_number: Maximum number of epochs to train
        patience: Number of epochs to wait for improvement before stopping
    
    Returns:
        best_valid_loss: Best validation loss achieved during training
    """
    N_EPOCHS = epoch_number
    consecutive_no_improvement = 0

    best_valid_loss = float("inf")  # Initialize with infinity (any loss will be better)

    # Store losses for plotting
    train_losses = []
    valid_losses = []

    # We loop forward on the epoch number
    for epoch in range(N_EPOCHS):

        start_time = time.time()

        # Train and evaluate
        train_loss, train_prec, train_rec, train_fscore = train(
            model, train_iterator, optimizer, criterion
        )
        
        valid_loss, valid_prec, valid_rec, valid_fscore = evaluate(
            model, valid_iterator, criterion
        )

        end_time = time.time()

        epoch_mins, epoch_secs = epoch_time(start_time, end_time)

        # Store losses
        train_losses.append(train_loss)
        valid_losses.append(valid_loss)

        # Check if validation loss improved
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            consecutive_no_improvement = 0 # Reset counter
            torch.save(model.state_dict(), "bow-deep-model.pt")

        else:
            consecutive_no_improvement += 1 # Increment counter

        # Print epoch results
        print(f"Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s")
        print(
            f"\tTrain Loss: {train_loss:.3f} | Train Prec: {train_prec*100:.2f}% | Train Rec: {train_rec*100:.2f}% | Train Fscore: {train_fscore*100:.2f}%"
        )
        print(
            f"\t Val. Loss: {valid_loss:.3f} |  Val Prec: {valid_prec*100:.2f}% | Val Rec: {valid_rec*100:.2f}% | Val Fscore: {valid_fscore*100:.2f}%"
        )

        # Early stopping check
        if consecutive_no_improvement >= patience:
            print(f"No improvement in validation loss for {patience} consecutive epochs. Early stopping after epoch {epoch+1}.")
            break  # Terminate training loop

    plot_losses(train_losses, valid_losses, n_epochs=epoch+1)
        
    return best_valid_loss

In [ ]:
training_loop()

## 8. Handling class imbalance

Our data is imbalanced. The NOT class (0) has roughly twice as many samples as the OFF class (1). This imbalance can bias the model toward predicting the majority class.


In [ ]:
tr_data.groupby("label").size()

### Solution: Weighted Loss Function

We can penalize errors on the minority class more heavily by adding weights to our loss function. The idea is to weight each class by the inverse of its frequency.


In [ ]:
# Class 1 (offensive) appears half as often, so weight it twice as much
weights = torch.Tensor([1, 2])
criterion = nn.NLLLoss(weight=weights)

In [ ]:
model = BoWDeepClassifier(OUTPUT_DIM, INPUT_DIM, HIDDEN_SIZE)
model = model.to(device)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.NLLLoss(weight=weights)

# Train
training_loop(epoch_number=15, patience=3)

## 9. Adding Embedding Layers

### Why Move Beyond Bag-of-Words?
- So far, we've only used Bag-of-Words representations (size 3000) as features. Limitations:
- **Sparse:** Most values are zero
- **No word relationships:** "good" and "great" are treated as completely different
- **Large dimensionality:** 3000 features for 3000 vocabulary words

### Enter: Word Embeddings
**Key idea:** Represent each word as a dense vector where similar words have similar vectors.

- Words are mapped to continuous vectors (e.g., 100 dimensions instead of 3000)
- The network learns these representations during training
- Semantically similar words end up close together in vector space

We'll use PyTorch's [Embedding layer](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html) which learns a dense vector for each word in our vocabulary.

![Embedding architecture](https://pytorch.org/tutorials/_images/text_sentiment_ngrams_model.png)
*Architecture with embedding layer (from pytorch.org)*


### Prepare Data: Convert Text to Token IDs

First, we need to convert words to numerical IDs that the embedding layer can process.

In [ ]:
# Get the analyzer from CountVectorizer (handles tokenization, preprocessing, stopword removal)
an = word_to_ix.build_analyzer() 

In [ ]:
# Example: tokenize and process a sentence
an("What made labubu go viral?")

In [ ]:
# Our vocabulary maps tokens to IDs
print(f"Vocabulary size: {len(word_to_ix.vocabulary_)}")
print(f"Example mapping: 'hello' -> {word_to_ix.vocabulary_.get('hello', 'not in vocab')}")

In [ ]:
def create_input(dataset, analyzer, vocabulary):
    """
    Convert text to sequences of token IDs.
    
    Args:
        dataset: List of text samples
        analyzer: Tokenizer function from CountVectorizer
        vocabulary: Word-to-ID mapping
    
    Returns:
        List of tensors, each containing token IDs for one sample
    
    Special tokens:
        - IDs 0-2999: Vocabulary words
        - ID 3000:  (unknown words not in vocabulary)
        - ID 3001:  (padding, added later)
    """

    dataset_as_indices = []

    for tweet in dataset:
        tokens = analyzer(tweet)
        token_ids = []

        for token in tokens:
            if token in vocabulary:
                token_ids.append(vocabulary[token]) # Known word
            else:
                token_ids.append(3000) # Unknown token

        # If all tokens were removed (stopwords, etc.), add unknown token
        if not token_ids:
            token_ids.append(3000)
        dataset_as_indices.append(torch.LongTensor(token_ids).to(device))

    return dataset_as_indices

In [ ]:
# We create the dataset as ids of tokens
dataset_as_ids = create_input(tr_data.tweet, an, word_to_ix.vocabulary_)
dataset_as_ids

In [ ]:
# Example: view token IDs for a sample
print(f"Sample tweet: '{tr_data.iloc[90]['tweet']}'")
print(f"Token IDs: {dataset_as_ids[90]}")

## 10. Padding Sequences

**Problem:** Neural networks expect fixed-size inputs, but tweets have different lengths.

**Solution:** Pad shorter sequences with a special `<PAD>` token (ID 3001) to make all sequences the same length.

![Padding visualization](https://miro.medium.com/max/1218/1*zsIXWoN0_CE9PXzmY3tIjQ.png)
*Padding sequences to uniform length (image from towardsdatascience.com)*

In [ ]:
tr_data.tweet.str.len()

In [ ]:
# Sort data by length (groups similar-length sequences together for efficiency)
tr_data = tr_data.assign(length=tr_data.tweet.str.len()).sort_values(by='length')
val_data = val_data.assign(length=val_data.tweet.str.len()).sort_values(by='length')

In [ ]:
val_data

In [ ]:
from torch.nn.utils.rnn import pad_sequence

# Example: pad a list of sequences
sample_sequences = [
    torch.tensor([1, 2, 3]),
    torch.tensor([4, 5]),
    torch.tensor([6, 7, 8, 9])
]
padded = pad_sequence(sample_sequences, batch_first=True, padding_value=3001)
print(f"Original sequences:\n{sample_sequences}")
print(f"\nPadded sequences (batch_first=True):\n{padded}")
print(f"Shape: {padded.shape}  # [num_sequences, max_length]")

`torch.nn.utils.rnn.pad_sequence` takes a list of tensors as input (sequences), calculates max length, and then pads all the sequences with `padding_value` to make them all have the same length

Here you can see the dimension of `padded`, so max length among the sequences is 86.

In [ ]:
def prepare_dataloader_with_padding(tr_data, val_data, word_to_ix):
    """
    Convert text to padded token ID sequences and create data loaders.
    
    Returns tuples of (padded_sequences, labels, original_lengths, raw_texts)
    """
    
    tr_data_vecs = pad_sequence(
        create_input(tr_data.tweet, an, word_to_ix.vocabulary_),
        batch_first=True, # Shape: [batch_size, max_length]
        padding_value=3001,
    )
    tr_labels = torch.LongTensor(tr_data.label.tolist()).to(device)
    tr_lens = torch.LongTensor(
        [len(i) for i in create_input(tr_data.tweet, an, word_to_ix.vocabulary_)] #len of tweets
    )
    # Print information about sequence lengths (stored BEFORE padding for pack_padded_sequence)
    print(f"\nTraining sequences: {tr_data_vecs.shape[0]} samples padded to max length {tr_data_vecs.shape[1]}")
    print(f"Original lengths (before padding): min={tr_lens.min()}, max={tr_lens.max()}, mean={tr_lens.float().mean():.1f}")
    #print(f"Lengths tensor shape: {tr_lens.shape}")

    # We also add the texts to the batches
    # This is for the Transformer models, you won't need this in the next experiments
    tr_raw_texts = tr_data.tweet.tolist()

    val_data_vecs = pad_sequence(
        create_input(val_data.tweet, an, word_to_ix.vocabulary_),
        batch_first=True,
        padding_value=3001,
    )
    val_labels = torch.LongTensor(val_data.label.tolist()).to(device)
    val_lens = torch.LongTensor(
        [len(i) for i in create_input(val_data.tweet, an, word_to_ix.vocabulary_)]
    )

    val_raw_texts = val_data.tweet.tolist()

    # Create data loaders with 4 elements per sample
    tr_data_loader = [
        (sample, label, length, sent)
        for sample, label, length, sent in zip(
            tr_data_vecs, tr_labels, tr_lens, tr_raw_texts
        )
    ]
    val_data_loader = [
        (sample, label, length, sent)
        for sample, label, length, sent in zip(
            val_data_vecs, val_labels, val_lens, val_raw_texts
        )
    ]

    return tr_data_loader, val_data_loader

In [ ]:
# Prepare padded data
tr_data_loader, val_data_loader = prepare_dataloader_with_padding(
    tr_data, val_data, word_to_ix
)

In [ ]:
def create_dataloader_iterators_with_padding(
    tr_data_loader, val_data_loader, BATCH_SIZE
):
    train_iterator = DataLoader(
        tr_data_loader,
        batch_size=BATCH_SIZE,
        shuffle=True,
    )

    valid_iterator = DataLoader(
        val_data_loader,
        batch_size=BATCH_SIZE,
        shuffle=False,
    )

    return train_iterator, valid_iterator

In [ ]:
# Create DataLoader iterators
train_iterator, valid_iterator = create_dataloader_iterators_with_padding(
    tr_data_loader, val_data_loader, BATCH_SIZE
)

**Inspect a batch:**

In [ ]:
padded_samples, labels, orig_lengths, raw_texts = next(iter(train_iterator))
print(f'Dimension of tensors with data samples (BATCH_SIZE, max_sequence_length): {padded_samples.shape}')
print(f'Number of tweets in batch: {len(raw_texts)}')
print(f'\nExample (2nd sample in batch):')
print(f'  Token IDs (padded): {padded_samples[1]}')
print(f'  Original length: {orig_lengths[1]} tokens')
print(f"  Label: {labels[1].item()} ({'OFF' if labels[1].item() == 1 else 'NOT'})")
print(f'  Raw text: "{raw_texts[1]}"')

## 11. Building the Embedding-Based Model

### Architecture

![Embedding + Pooling architecture](https://github.com/bentrevett/pytorch-sentiment-analysis/raw/b4efbefa47672174394a8b6a27d4e7bc193bc224/assets/sentiment8.png)

*Bag-of-Words model with embedding layer and max pooling (from bentrevett)*

**How it works:**
1. **Embedding layer:** Maps each token ID to a dense vector
2. **Max pooling:** Reduces the sequence of word embeddings to a single vector by taking the maximum value across each dimension
3. **Linear layer:** Maps the pooled vector to class scores

**Max pooling example:**
```
Word embeddings:
  "I"    → [0.1, 0.3, 0.2]
  "hate" → [0.5, 0.7, 0.6]
  "this" → [0.2, 0.1, 0.3]
  "film" → [0.4, 0.9, 0.8]

Max pooling (take max of each column):
  Result → [0.5, 0.9, 0.8]  ← This represents the entire tweet

In [ ]:
from torch import nn
import numpy as np


class BoWClassifierWithEmbedding(nn.Module):
    """Bag-of-Words classifier using learned embeddings and max pooling."""
    def __init__(self, num_labels, vocab_size, embedding_dim):
        super(BoWClassifierWithEmbedding, self).__init__()

        # Embedding layer: maps token IDs to dense vectors
        # vocab_size: total number of words (including  and )
        # embedding_dim: size of each word vector
        # padding_idx: ID for padding token (always maps to zeros, not trained)
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=3001)
        # Enable training of embeddings (can be set to False to use pre-trained embeddings)
        self.embedding.weight.requires_grad = True
        # Linear layer: maps pooled embedding to class scores
        self.linear = nn.Linear(embedding_dim, num_labels)

    def forward(self, text, sequence_lens):
        """
        Forward pass.
        
        Args:
            text: Token IDs [batch_size, seq_len]
            sequence_lens: Original sequence lengths (not used here)
        
        Returns:
            Log probabilities [batch_size, num_labels]
        """
        # Convert token IDs to embeddings
        # Shape: [batch_size, seq_len] → [batch_size, seq_len, embedding_dim]
        embedded = self.embedding(text)  
        # Max pooling: reduce sequence to single vector
        # For each of the embedding_dim dimensions, take the max value across all words
        # kernel_size=(seq_len, 1) means: max over all words, separately for each embedding dimension
        # Shape: [batch_size, seq_len, embedding_dim] → [batch_size, embedding_dim]
        pooled = nn.functional.max_pool2d(embedded, (embedded.shape[1], 1)).squeeze(1)  # dim=64x100 (batch_size x embedding_dim)
        # Pass through linear layer and apply log_softmax
        return nn.functional.log_softmax(self.linear(pooled), dim=1)

In [ ]:
# Initialize model
INPUT_DIM = VOCAB_SIZE + 2  # 3002
OUTPUT_DIM = 2
EMBEDDING_DIM = 100  # Each word represented as a 100-dimensional vector
HIDDEN_DIM = 20
criterion = nn.NLLLoss()

model = BoWClassifierWithEmbedding(OUTPUT_DIM, INPUT_DIM, EMBEDDING_DIM)
model = model.to(device)

In [ ]:
# Set up training
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.NLLLoss()
criterion = criterion.to(device)

print(model)
print(f"Total parameters: {sum(p.numel() for p in model.parameters())}")
print(f"Embedding parameters: {model.embedding.weight.numel()}")

In [ ]:
### Train the Model
training_loop(epoch_number=15)

> **Note:** The embedding layer adds many parameters (vocab_size × embedding_dim = 3002 × 100 = 300,200). This gives the model more capacity to learn, but also increases the risk of overfitting on small datasets. Watch the validation loss carefully!

***Optional: Using Pre-trained Embeddings***

Instead of learning embeddings from scratch, you can load pre-trained embeddings like GloVe or Word2Vec.

## 12. Standard RNNs & LSTMs

### Why Sequential Models?

So far, our models have ignored word order:
- **Bag-of-Words:** Treats "I love this movie" and "This movie I love" identically
- **Max pooling:** Takes the maximum embedding value, losing sequence information

**RNNs process text sequentially**, maintaining a hidden state that captures information from previous words. This allows them to understand context and word order.

### Understanding RNNs

An RNN processes one word at a time, updating its hidden state at each step:

![Unrolled RNN](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png)
*Unrolled RNN showing sequential processing (from [colah's blog](https://colah.github.io/posts/2015-08-Understanding-LSTMs/))*

**Standard RNN module:**

![RNN module](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-SimpleRNN.png)
*A single RNN cell applies a simple transformation (from [colah's blog](https://colah.github.io/posts/2015-08-Understanding-LSTMs/))*

**Problem with standard RNNs:** They struggle to learn long-term dependencies due to vanishing/exploding gradients.

### Long Short-Term Memory (LSTM)

LSTMs solve the long-term dependency problem using a more sophisticated cell structure with gates that control information flow:

![LSTM module](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png)
*LSTM cell with gates for controlling information (from [colah's blog](https://colah.github.io/posts/2015-08-Understanding-LSTMs/))*

**LSTM outputs:**

![LSTM outputs](https://i.stack.imgur.com/SjnTl.png)

*LSTM produces hidden states (h) and cell states (c) at each time step (from stackoverflow)*


In [ ]:
class LSTMClassifier(nn.Module):
    """
    Text classifier using LSTM for sequential processing.
    
    Architecture: Embeddings → LSTM → Linear → Output
    The LSTM processes the sequence and we use its final hidden state for classification.
    """
    def __init__(self, num_labels, vocab_size, embedding_dim, hidden_dim):
        super(LSTMClassifier, self).__init__()
        # Embedding layer (same as before)
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=3001)
        self.embedding.weight.requires_grad = True

        # LSTM layer
        # Takes sequence of embeddings, outputs sequence of hidden states
        # Documentation: https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html
        self.lstm = nn.LSTM(
            embedding_dim,          # Size of input embeddings
            hidden_dim,             # Size of hidden state
            batch_first=True,       # Input shape: [batch, seq_len, features]
            num_layers=1,           # Single LSTM layer
            bidirectional=False,    # Process sequence left-to-right only
        )
        # Classification layer (uses final LSTM hidden state)
        self.linear = nn.Linear(hidden_dim, num_labels)
        # Dropout for regularization to overcome overfitting
        self.dropout = nn.Dropout(0.25)

    def forward(self, text, sequence_lens):
        """
        Forward pass.
        
        Args:
            text: Padded token IDs [batch_size, max_seq_len]
            sequence_lens: Original sequence lengths (before padding)
        
        Returns:
            Log probabilities [batch_size, num_labels]
        """
        # Convert token IDs to embeddings
        # Shape: [batch_size, seq_len, embedding_dim]
        embedded = self.embedding(text)

        # Pack padded sequences to avoid computing LSTM on padding tokens
        # This is more efficient and prevents the LSTM from learning patterns in padding
        packed = nn.utils.rnn.pack_padded_sequence(
            embedded,
            sequence_lens,
            enforce_sorted=False,   # Don't require sequences sorted by length
            batch_first=True
        )
        # Pass through LSTM
        # packed_outputs: all hidden states (one per time step)
        # h: final hidden state [num_layers, batch_size, hidden_dim]
        # c: final cell state (not used for classification)
        packed_outputs, (h, c) = self.lstm(packed)
        # Unpack the outputs (converts back to padded format)
        # We don't actually use lstm_outputs here, but this shows how to extract them
        lstm_outputs, lens = nn.utils.rnn.pad_packed_sequence(
            packed_outputs, batch_first=True
        )

        # Use the final hidden state for classification
        # h[-1] selects the last layer's hidden state
        # Shape: [batch_size, hidden_dim]
        final_hidden = h[-1]
        
        # Apply dropout and linear transformation
        output = self.linear(self.dropout(final_hidden))
        
        # Convert to log probabilities
        return nn.functional.log_softmax(output, dim=1)

### Train the LSTM Model

In [ ]:
INPUT_DIM = VOCAB_SIZE + 2  # 3000 +  + 
OUTPUT_DIM = 2
EMBEDDING_DIM = 100
HIDDEN_DIM = 20  # Size of LSTM hidden state

# Initialize model
model = LSTMClassifier(OUTPUT_DIM, INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM)
model = model.to(device)

In [ ]:
# Set up training
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.NLLLoss()
criterion = criterion.to(device)

print(model)
print(f"Total parameters: {sum(p.numel() for p in model.parameters())}")

In [ ]:
training_loop(epoch_number=15)

### Understanding pack_padded_sequence

**Why pack sequences?**
- Padded tokens (ID 3001) don't contain useful information
- Without packing, the LSTM would waste computation on padding
- Packing tells the LSTM where each sequence actually ends

### Further Reading: Advanced LSTMs

Interested in the latest LSTM developments? Check out:

**[xLSTM: Extended Long Short-Term Memory](https://arxiv.org/abs/2405.04517)** by Sepp Hochreiter's group (2024)

Key innovations:
- **Exponential gating:** Improved information flow control
- **New memory structures:** Better handling of long-term dependencies
- **mLSTM variant:** Addresses parallelization limitations of traditional LSTMs

## 13. Transformers

### Why Transformers? Problems with RNNs

Recall that we used LSTMs to encode sequences into vectors. While effective, RNNs have fundamental limitations:

**Problem 1: No Parallelism**
- LSTMs process tokens sequentially (left-to-right), requiring previous steps to complete before processing the next token
- Cannot parallelize computation across the sequence

**Problem 2: Long-Range Dependencies**
- Despite improvements over vanilla RNNs, LSTMs still struggle with very long-range dependencies
- Example: "The people who called and wanted to rent your house when you go away next year are from California"
- Too many backpropagation steps between distant related words

![Encoder-Decoder RNN](https://miro.medium.com/v2/resize:fit:720/format:webp/1*1JcHGUU7rFgtXC_mydUA_Q.jpeg)
*Traditional seq2seq without attention (from [medium](https://towardsdatascience.com/understanding-encoder-decoder-sequence-to-sequence-model-679e04af4346))*

### The Solution: Attention Mechanism

The attention mechanism, introduced in 2014 ([Bahdanau et al.](https://arxiv.org/abs/1409.0473)), revolutionized NLP by allowing models to focus on relevant parts of the input when making predictions.

![Encoder-Decoder with Attention](https://miro.medium.com/v2/resize:fit:640/format:webp/1*KrCUK30Y1R7TV7xE-ltcpg.png)
*Seq2seq with attention mechanism (from [medium](https://medium.com/data-science-community-srm/understanding-encoders-decoders-with-attention-based-mechanism-c1eb7164c581))*

**Key innovation:** Instead of compressing the entire input into a single vector, attention allows the model to look back at all input tokens when making predictions.

### Transformers: "Attention Is All You Need"

The [Transformer architecture](https://arxiv.org/abs/1706.03762) (Vaswani et al., 2017) eliminates recurrence entirely, relying solely on attention mechanisms.

![Transformer Architecture](https://machinelearningmastery.com/wp-content/uploads/2021/08/attention_research_1.png)
*Transformer architecture (from [machinelearningmastery.com](https://machinelearningmastery.com))*
**Key features:**
- **Self-attention:** Each word attends to all other words in the sequence
- **Parallel processing:** No sequential dependencies, can process entire sequence at once
- **Multi-headed attention:** Multiple attention mechanisms running in parallel

**Self-attention example:**
"The animal didn't cross the street because **it** was too tired"
- Self-attention learns to associate "it" with "animal"

![Self-attention mechanism](https://miro.medium.com/v2/resize:fit:4800/format:webp/1*maneqIvralwRZWf0g3hS5w.png)
*Self-attention visualization (from [towardsdatascience](https://towardsdatascience.com/all-you-need-to-know-about-attention-and-transformers-in-depth-understanding-part-1-552f0b41d021))*

**Learn more:** [The Illustrated Transformer](https://jalammar.github.io/illustrated-transformer/) by Jay Alammar

### BERT: Bidirectional Encoder Representations from Transformers

[BERT](https://www.aclweb.org/anthology/N19-1423/) (Devlin et al., 2018) is a pre-trained transformer model that revolutionized NLP by enabling transfer learning.

**Pre-training objectives:**
1. **Masked Language Model (MLM):** Predict randomly masked words
   - 15% of tokens are masked: 80% replaced with [MASK], 10% random token, 10% unchanged
2. **Next Sentence Prediction (NSP):** Predict if two sentences are consecutive

**Training scale:**
- BERT-base: 110M parameters, trained on BookCorpus + English Wikipedia
- Vocabulary size: 30,522 WordPiece tokens

**Fine-tuning approach:**
1. Load pre-trained BERT model
2. Add task-specific classification layer on top
3. Fine-tune on your labeled dataset (much smaller than pre-training data)
4. Option: Freeze BERT weights and train only classification layer

![BERT fine-tuning](https://production-media.paperswithcode.com/methods/new_BERT_Overall.jpg)
*BERT fine-tuning for different tasks*

### WordPiece Tokenization

BERT uses its own tokenizer - **always tokenize with BERT's tokenizer, not your custom preprocessing!**

**Special tokens:**
- `[CLS]`: Classification token (start of sequence)
- `[SEP]`: Separator token (end of sequence)
- `[MASK]`: Masked token (for pre-training)
- `[UNK]`: Unknown token
- `[PAD]`: Padding token (ID 0)

**How it works:** Subword tokenization between character and word level
- Frequent words: kept whole
- Rare words: split into subwords
- Unknown characters: handled gracefully

In [ ]:
# Load tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Example tokenization
text = "I love Python 🐍 and data analysis 📊!"
tokens = tokenizer.tokenize(text)
print(f"Tokens: {tokens}")

# Check vocabulary size
print(f"Vocabulary size: {len(tokenizer.get_vocab())}")

In [ ]:
# Example: complex name tokenization
tokens = tokenizer.tokenize("His full name is Fyodor Mikhailovich Dostoevsky")
print(f"Name tokens: {tokens}")

**Tokenize two sentences:**

In [ ]:
# Format: [CLS] Sentence 1 [SEP] Sentence 2 [SEP]
encoded = tokenizer("There are black cats and black dogs.", "Another sentence.")
print(encoded)

=> [CLS] Sentence 1 [SEP] Sentence 2 [SEP]

100: [UNK] (Unknown token)

101: [CLS] (Classification Token)

102: [SEP] (Separator Token): The [CLS] token is special in BERT as it is used to aggregate information from the entire sequence. Its output embedding (from the last layer of the model) is often used for classification tasks (e.g., sentiment analysis, sentence-pair classification).

103: [MASK] (Masked Token)

0: [PAD] (Padding)


In [ ]:
print(type(tokenizer))
print(len(tokenizer.get_vocab()))

tokenizer.tokenize("His full name is Fyodor Mikhailovich Dostoevsky")

**Now, let's have a look at how another tokeniser works:**

In [ ]:
from transformers import BertTokenizer

# BERTweet (trained on tweets)
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base")
text = "I love Python 🐍 and data analysis 📊!"

# Tokenize the text
tokens = tokenizer.tokenize(text)
print(tokens)

### Fine-tuning BERT on Our Dataset with HuggingFace

**Prepare data:**

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Convert dataframes to lists
train_texts = tr_data.tweet.tolist()
train_labels = tr_data.label.tolist()
val_texts = val_data.tweet.tolist()
val_labels = val_data.label.tolist()

In [ ]:
# Tokenize all at once
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

**Create PyTorch Dataset:**

In [ ]:
class TweetDataset(torch.utils.data.Dataset):
    """Custom dataset for BERT fine-tuning."""
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TweetDataset(train_encodings, train_labels)
val_dataset = TweetDataset(val_encodings, val_labels)

**Define metrics:**

In [ ]:
def compute_metrics(pred):
    """Compute metrics for evaluation."""
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average='binary', pos_label=1
    )
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

**Train with Weights & Biases tracking:**


In [ ]:
# Initialize W&B
current_date = datetime.now().strftime("%Y%m%d")
wandb.init(project="bert-finetuning", name=f"bert-base-uncased-semeval-{current_date}")

In [ ]:
# Training configuration
training_args = TrainingArguments(
    output_dir=f"./results_bert_{current_date}",
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=5e-5,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir=f"./logs_bert_{current_date}",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    greater_is_better=False,
    report_to="wandb"
)

In [ ]:
# Initialize model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# Train
trainer.train()

# Final evaluation
results = trainer.evaluate()
print(f"\nFinal results: {results}")
wandb.log(results)

# Save model
trainer.save_model(f"./saved_model_bert_{current_date}")

# Finish W&B run
wandb.finish()